# Example of workflow with the EnergyScope model

In [1]:
# Import the required libraries
from mescal import *
import pandas as pd
import bw2data as bd

In [2]:
ecoinvent_version = '3.10' # choose the ecoinvent version you wish to use
esm_location = 'CH' # choose the version of energyscope for which you want to generate metrics
esm_year = 2050 # choose the year of the energyscope snapshot model or transition pathway time-step

In [3]:
# Set up your Brightway project
bd.projects.set_current(f'ecoinvent{ecoinvent_version}')  # put the name of your brightway project here

In [4]:
mapping = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}.csv')
unit_conversion = pd.read_csv(f'../dev/energyscope_data/{esm_location}/unit_conversion_{ecoinvent_version}.csv')
mapping_esm_flows_to_CPC = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_esm_flows_to_CPC.csv')
model = pd.read_csv(f'../dev/energyscope_data/{esm_location}/model.csv')
technology_compositions = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_compositions.csv')
technology_specifics = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_specifics.csv') 
lifetime = pd.read_csv(f'../dev/energyscope_data/{esm_location}/lifetime.csv')
mapping_product_to_CPC = pd.read_csv('../dev/data/mapping_product_to_CPC.csv')
impact_abbrev = pd.read_csv('../dev/IW+/impact_abbrev.csv')
technologies_to_remove_from_layers = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technologies_to_remove_from_layers.csv')
new_end_use_types = pd.read_csv(f'../dev/energyscope_data/{esm_location}/new_end_use_types.csv')
premise_changes = pd.read_csv('../dev/data/premise_change_report.csv')

## Add CPC categories to the premise database

In [5]:
name_premise_db = f"ecoinvent_cutoff_{ecoinvent_version}_remind_SSP2-Base_{esm_year}"
name_premise_with_CPC_db = name_premise_db + '_with_CPC'

In [33]:
premise_db = load_extract_db(name_premise_db, create_pickle=True)

Getting activity data


100%|██████████| 34093/34093 [00:06<00:00, 5452.81it/s] 


Adding exchange data to activities


100%|██████████| 1154986/1154986 [01:46<00:00, 10798.52it/s]


Filling out exchange data


100%|██████████| 34093/34093 [00:06<00:00, 5100.88it/s]


ecoinvent_cutoff_3.10_remind_SSP2-Base_2050.pickle created!


In [34]:
create_new_database_with_CPC_categories(db=premise_db, new_db_name=name_premise_with_CPC_db, mapping_product_to_CPC=mapping_product_to_CPC)

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:02


Title: Writing activities to SQLite3 database:
  Started: 07/09/2024 17:16:45
  Finished: 07/09/2024 17:18:47
  Total time elapsed: 00:02:02
  CPU %: 32.30
  Memory %: 37.12


## Relinking the mapping file with a premise database

In [6]:
premise_db_with_CPC = load_extract_db(name_premise_with_CPC_db, create_pickle=True)

In [7]:
mapping = change_mapping_year(mapping, 2020, esm_year)

In [8]:
test_mapping_file(mapping, premise_db_with_CPC)

Mapping successfully linked to the database


In [9]:
mapping_linked_to_premise = create_complementary_database(mapping, premise_db_with_CPC, name_premise_db+f'_comp_{esm_location}', premise_changes)

The complementary database did not have to be created


In [10]:
if name_premise_db+f'_comp_{esm_location}' in bd.databases:
    premise_comp_db = load_extract_db(name_premise_db+f'_comp_{esm_location}') # complementary database without CPC categories
    create_new_database_with_CPC_categories(db=premise_comp_db, new_db_name=name_premise_db+f'_comp_{esm_location}', mapping_product_to_CPC=mapping_product_to_CPC)
    premise_comp_db = load_extract_db(name_premise_db+f'_comp_{esm_location}') # complementary database with CPC categories
    main_db = premise_db_with_CPC + premise_comp_db
else:
    main_db = premise_db_with_CPC

## Change location according to user ranking

In [11]:
# Define the user-defined ranking
if esm_location == 'CA-QC':
    my_ranking = [
        'CA-QC', # Quebec
        'CA', # Canada
        'CA-ON', # Other canadian provinces 
        'CA-AB',
        'CA-BC',
        'CA-MB',
        'CA-NB',
        'CA-NF',
        'CA-NS',
        'CA-NT',
        'CA-NU',
        'CA-PE',
        'CAZ', # Canada - Australia - New Zealand
        'RNA', # North America
        'US', # United States
        'USA', # United States
        'GLO', # Global average 
        'RoW', # Rest of the world
    ]
elif esm_location == 'CH':
    my_ranking = [
        'CH', 
        'RER', 
        'IAI Area, EU27 & EFTA',
        'NEU',
        'EUR',
        'GLO',
        'RoW'
    ]
else:
    my_ranking = [
        'GLO',
        'RoW',
    ]

In [12]:
# sufficient match within ecoinvent
if esm_location == 'CA-QC':
    accepted_locations = ['CA-QC', 'CA']  
elif esm_location == 'CH':
    accepted_locations = ['CH'] 
else:
    accepted_locations = ['GLO', 'RoW'] 

In [13]:
# Update mapping dataframe
mapping_linked_to_premise = change_location_mapping_file(mapping_linked_to_premise, my_ranking, premise_db_with_CPC, esm_location)

In [14]:
mapping_linked_to_premise.head()

,Name,Type,Product,Activity,Location,Database
0,ALKALINE_ELECTROLYSIS,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.10_remind_SSP2-Base_2050_wi...
1,ALKALINE_ELECTROLYSIS_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.10_remind_SSP2-Base_2050_wi...
2,ALKALINE_ELECTROLYSIS_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.10_remind_SSP2-Base_2050_wi...
3,ALKALINE_ELECTROLYSIS_STACK,Construction,"electrolyzer, 1MWe, AEC, Stack","electrolyzer production, 1MWe, AEC, Stack",RER,ecoinvent_cutoff_3.10_remind_SSP2-Base_2050_wi...
4,ALKALINE_ELECTROLYSIS_STACK_DECOM,Construction,"used fuel cell stack, 1MWe, AEC","treatment of fuel cell stack, 1MWe, AEC",RER,ecoinvent_cutoff_3.10_remind_SSP2-Base_2050_wi...


## Perform double-counting removal

In [15]:
mapping_linked_to_premise = mapping_linked_to_premise[mapping_linked_to_premise['Type'] != 'Flow']

In [16]:
new_db_name = f'energyscope_{esm_location}_{esm_year}'

In [17]:
regionalize_foregrounds = True # Set to True if you want to regionalize foregrounds

In [18]:
mapping_linked_to_premise_new_code = create_esm_database(
    mapping=mapping_linked_to_premise,
    model=model,
    tech_specifics=technology_specifics,
    technology_compositions=technology_compositions,
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC,
    main_database=main_db,
    esm_db_name=new_db_name,
    regionalize_foregrounds=regionalize_foregrounds,
    accepted_locations=accepted_locations,
    target_region=esm_location,
    locations_ranking=my_ranking,
    results_path_file=f'results/energyscope_{esm_location}/{esm_year}/',
)

ALKALINE_ELECTROLYSIS
AL_MAKING
AL_MAKING_HR
AN_DIG
AN_DIG_SI
BIOMASS_ETHANOL
BUS_CNG_STOICH
BUS_DIESEL
BUS_EV
BUS_FC_HYBRID_CH4
BUS_FC_HYBRID_H2
BUS_HY_DIESEL
CAR_BEV_LOWRANGE
CAR_BEV_MEDRANGE_LOCAL
CAR_BEV_MEDRANGE_LONGD
CAR_DIESEL_LOCAL
CAR_DIESEL_LONGD
CAR_DME_D10_LOCAL
CAR_DME_D10_LONGD
CAR_ETOH_E10_LOCAL
CAR_ETOH_E10_LONGD
CAR_ETOH_E85_LOCAL
CAR_ETOH_E85_LONGD
CAR_FC_CH4_LOCAL
CAR_FC_CH4_LONGD
CAR_FC_H2_LOCAL
CAR_FC_H2_LONGD
CAR_GASOLINE_LOCAL
CAR_GASOLINE_LONGD
CAR_HEV_LOCAL
CAR_HEV_LONGD
CAR_MEOH_LOCAL
CAR_MEOH_LONGD
CAR_NG_LOCAL
CAR_NG_LONGD
CAR_PHEV_LOCAL
CAR_PHEV_LONGD
CCGT
CCGT_CC
CEMENT_PROD
CEMENT_PROD_HP
CO2_METHANOL
CO2_TO_DIESEL
CO2_TO_JETFUELS
COACH_CNG_STOICH
COACH_DIESEL
COACH_EV
COACH_FC_HYBRID_CH4
COACH_FC_HYBRID_H2
COACH_HY_DIESEL
COAL_IGCC
COAL_IGCC_CC
COAL_US
COAL_US_CC
COMMUTER_RAIL_DIESEL
COMMUTER_RAIL_ELEC
CROPS_TO_ETHANOL
CROPS_TO_JETFUELS
CUMENE_PROCESS
DAC_HT
DAC_LT
DEC_BOILER_GAS
DEC_BOILER_OIL
DEC_BOILER_WOOD
DEC_COGEN_GAS
DEC_COGEN_OIL
DEC_COGEN_WOOD
D

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/09/2024 18:10:53
  Finished: 07/09/2024 18:10:54
  Total time elapsed: 00:00:00
  CPU %: 32.60
  Memory %: 10.43


## Computing the LCA metrics

In [19]:
esm_db = load_extract_db(new_db_name)

Getting activity data


100%|██████████| 757/757 [00:00<00:00, 53978.82it/s]


Adding exchange data to activities


100%|██████████| 12114/12114 [00:01<00:00, 8782.26it/s] 


Filling out exchange data


100%|██████████| 757/757 [00:01<00:00, 535.59it/s]


In [20]:
R_long = compute_impact_scores(
    esm_db=esm_db,
    mapping=mapping_linked_to_premise_new_code,
    technology_compositions=technology_compositions,
    #methods=['IMPACT World+ Midpoint 2.0.1', 'IMPACT World+ Damage 2.0.1', 'IMPACT World+ Footprint 2.0.1'],
    methods=['PB LCIA'],
    unit_conversion=unit_conversion,
    lifetime=lifetime,
)

In [22]:
R_long.to_csv(f'results/energyscope_{esm_location}/{esm_year}/impact_scores.csv', index=False)

## Convert the results in AMPL format

In [22]:
refactor = 1e-2 # scaling factor to have the highest operation and construction metrics to the same order of magnitude (avoid having too many zeros)
lcia_method = 'IMPACT World+ Damage 2.0.1 - Total only'

### Create the .dat file

In [23]:
normalize_lca_metrics(
    R=R_long,
    f_norm=1e6,
    mip_gap=1e-6,
    refactor=refactor,
    lcia_method=lcia_method,
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/',
)

### Create the .mod file

In [24]:
gen_lcia_obj(
    lcia_method=lcia_method,
    refactor=refactor,
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/',
)